# Rock, Paper and Scissors

Game of stone, paper and scissors using a small neural network.

<b>Made by : Sergio Daniel Velasquez Pobre - Software engineer </b>


* Reference : https://www.youtube.com/watch?v=eZpSGS7vF5Y

In [59]:
options = ["rock", "scissors", "paper"]

In [60]:
def search_winner(p1, p2): # Method to find the winner
    if p1 == p2:
        result = 0
    
    elif p1 == "rock" and p2 == "scissors":
        result = 1
    elif p1 == "rock" and p2 == "paper":
        result = 2
    elif p1 == "scissors" and p2 == "rock":
        result = 2
    elif p1 == "scissors" and p2 == "paper":
        result = 1
    elif p1 == "paper" and p2 == "rock":
        result = 1
    elif p1 == "paper" and p2 == "scissors":
        result = 2
        
    return result

In [61]:
search_winner("paper", "scissors") # Demonstration of the method

2

In [62]:
# testing the game
test = [
    ["rock", "rock", 0],
    ["rock", "scissors", 1],
    ["rock", "paper", 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

player1: rock player2: rock Winner: 0 Validation: 0
player1: rock player2: scissors Winner: 1 Validation: 1
player1: rock player2: paper Winner: 2 Validation: 2


In [63]:
from random import choice
def get_choice():
    return choice(options) # choose an option within the list


In [64]:
for i in range(10):  # testing the game 
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: rock player2: scissors Winner: 1 
player1: scissors player2: scissors Winner: 0 
player1: scissors player2: paper Winner: 1 
player1: scissors player2: rock Winner: 2 
player1: paper player2: paper Winner: 0 
player1: scissors player2: paper Winner: 1 
player1: paper player2: rock Winner: 1 
player1: rock player2: scissors Winner: 1 
player1: rock player2: paper Winner: 2 
player1: rock player2: scissors Winner: 1 


In [65]:
def str_to_list(option): # Converting options into binary lists.
    if option=="rock":
        res = [1,0,0]
    elif option=="scissors":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, ["rock", "scissors", "paper"])) 
data_y = list(map(str_to_list, ["paper", "rock", "scissors"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [66]:
from sklearn.neural_network import MLPClassifier 
# from copy import deepcopy  # make a copy

In [67]:
clf = MLPClassifier(verbose=False, warm_start=True) #  implements a multi-layer perceptron (MLP) algorithm that trains using Backpropagation.

In [68]:
#tmp_clf = deepcopy(clf) # make a copy

In [69]:
#clf=tmp_clf # make a copy

In [70]:
model = clf.fit([data_X[0]], [data_y[0]]) # model training, it is only known that paper wins to stone
print(model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=True)


C:\Users\Amanda\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [71]:
def play_and_learn(iters=10, debug=False): 
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    #predictSumatoria=0
    
    for i in range(iters):
        player1 = get_choice()
        
        # you get the weight of each probability that occurs in each iteration
        predict = model.predict_proba([str_to_list(player1)])[0] 
        
        #predictSumatoria+=predict
        if predict[0] >= 0.85:
            player2 = options[0]
        elif predict[1] >= 0.85:
            player2 = options[1]
        elif predict[2] >= 0.85:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y #,predictSumatoria/iters


        

In [72]:
score, data_X, data_y= play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)


Player1: scissors Player2 (modelo): [ 0.1121821   0.11044454  0.8903142 ] --> paper
Comprobamos: p1 VS p2: 1
[]
[]
Score: {'win': 0, 'loose': 1} 0.0 %


In [73]:
i = 0
historic_pct = []
while True: 
    i+=1
    score, data_X, data_y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"])) # win*100/total <- percentage
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historic_pct[-9:])==900:
        break


Iter: 1 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 2 - score: {'win': 333, 'loose': 667} 33.3 %
Iter: 3 - score: {'win': 354, 'loose': 646} 35.4 %
Iter: 4 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 5 - score: {'win': 329, 'loose': 671} 32.9 %
Iter: 6 - score: {'win': 326, 'loose': 674} 32.6 %
Iter: 7 - score: {'win': 310, 'loose': 690} 31.0 %
Iter: 8 - score: {'win': 340, 'loose': 660} 34.0 %
Iter: 9 - score: {'win': 322, 'loose': 678} 32.2 %
Iter: 10 - score: {'win': 334, 'loose': 666} 33.4 %
Iter: 11 - score: {'win': 313, 'loose': 687} 31.3 %
Iter: 12 - score: {'win': 309, 'loose': 691} 30.9 %
Iter: 13 - score: {'win': 352, 'loose': 648} 35.2 %
Iter: 14 - score: {'win': 311, 'loose': 689} 31.1 %
Iter: 15 - score: {'win': 329, 'loose': 671} 32.9 %
Iter: 16 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 17 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 18 - score: {'win': 325, 'loose': 675} 32.5 %
Iter: 19 - score: {'win': 329, 'loose': 671} 32.9 %
Iter: 20 - score: {'w

Iter: 158 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 159 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 160 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 161 - score: {'win': 309, 'loose': 691} 30.9 %
Iter: 162 - score: {'win': 343, 'loose': 657} 34.3 %
Iter: 163 - score: {'win': 333, 'loose': 667} 33.3 %
Iter: 164 - score: {'win': 325, 'loose': 675} 32.5 %
Iter: 165 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 166 - score: {'win': 350, 'loose': 650} 35.0 %
Iter: 167 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 168 - score: {'win': 305, 'loose': 695} 30.5 %
Iter: 169 - score: {'win': 325, 'loose': 675} 32.5 %
Iter: 170 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 171 - score: {'win': 341, 'loose': 659} 34.1 %
Iter: 172 - score: {'win': 349, 'loose': 651} 34.9 %
Iter: 173 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 174 - score: {'win': 315, 'loose': 685} 31.5 %
Iter: 175 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 176 - score: {'win': 340, 'loose': 660} 

Iter: 312 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 313 - score: {'win': 322, 'loose': 678} 32.2 %
Iter: 314 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 315 - score: {'win': 312, 'loose': 688} 31.2 %
Iter: 316 - score: {'win': 316, 'loose': 684} 31.6 %
Iter: 317 - score: {'win': 326, 'loose': 674} 32.6 %
Iter: 318 - score: {'win': 344, 'loose': 656} 34.4 %
Iter: 319 - score: {'win': 326, 'loose': 674} 32.6 %
Iter: 320 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 321 - score: {'win': 296, 'loose': 704} 29.6 %
Iter: 322 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 323 - score: {'win': 330, 'loose': 670} 33.0 %
Iter: 324 - score: {'win': 340, 'loose': 660} 34.0 %
Iter: 325 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 326 - score: {'win': 344, 'loose': 656} 34.4 %
Iter: 327 - score: {'win': 368, 'loose': 632} 36.8 %
Iter: 328 - score: {'win': 332, 'loose': 668} 33.2 %
Iter: 329 - score: {'win': 313, 'loose': 687} 31.3 %
Iter: 330 - score: {'win': 345, 'loose': 655} 

KeyboardInterrupt: 

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()


In [57]:
x = range(len(historic_pct)) # List of iters
y = historic_pct # List of percentages

p = figure(
    title="Percentage of learning in each iteration",
    x_axis_label="Iterations", y_axis_label="Success percentage %", width=600)

p.line(x, y, legend=None, line_width=3,color="red")
show(p)


In [58]:
model.predict_proba([str_to_list("paper")]) # Choice probability, Scissors[0,1,0]

array([[ 0.04805665,  0.9526019 ,  0.02346198]])